# Features Store와 Amazon SageMaker XGBoost를 활용한 다이렉트 마케팅 타겟팅
_**그래디언트 부스팅 트리를 활용한 지도학습: 불균형 클래스를 가진 이진 예측 문제**_

---

---

## 목차

1. [배경](#Background)
1. [준비](#Preparation)
1. [데이터](#Data)
    1. [탐색](#Exploration)
    1. [변환](#Transformation)
1. [훈련](#Training)
1. [호스팅](#Hosting)
1. [평가](#Evaluation)
1. [확장](#Extensions)

---

## 배경
우편, 이메일, 전화 등을 통한 다이렉트 마케팅은 고객을 확보하는 일반적인 전략입니다. 자원과 고객의 관심이 제한적이기 때문에, 특정 제안에 참여할 가능성이 높은 잠재 고객의 하위 집합만을 타겟팅하는 것이 목표입니다. 인구통계학적 정보, 과거 상호작용, 환경적 요인과 같이 쉽게 이용 가능한 정보를 바탕으로 잠재 고객을 예측하는 것은 일반적인 머신러닝 문제입니다.

이 노트북은 한 번 이상의 전화 통화 후 고객이 은행의 정기예금에 가입할지를 예측하는 예제 문제를 제시합니다. 단계는 다음과 같습니다:

* Amazon SageMaker 노트북 준비
* 인터넷에서 Amazon SageMaker로 데이터 다운로드
* Amazon SageMaker 알고리즘에 입력할 수 있도록 데이터 조사 및 변환
* 그래디언트 부스팅 알고리즘을 사용한 모델 추정
* 모델의 효과성 평가
* 지속적인 예측을 위한 모델 설정

---

## 준비

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성되고 테스트되었습니다._

다음을 지정하는 것부터 시작하겠습니다:

- 훈련 및 모델 데이터에 사용할 S3 버킷과 접두사. 이는 노트북 인스턴스, 훈련, 호스팅과 동일한 리전 내에 있어야 합니다.
- 훈련 및 호스팅이 데이터에 액세스할 수 있도록 하는 IAM 역할 arn. 이를 생성하는 방법은 문서를 참조하세요. 노트북 인스턴스, 훈련, 호스팅에 둘 이상의 역할이 필요한 경우, boto regexp를 적절한 전체 IAM 역할 arn 문자열로 교체하세요.

In [ ]:
# cell 01
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

이제 분석 전반에 걸쳐 사용할 Python 라이브러리들을 가져오겠습니다

In [ ]:
# cell 02
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

**S3에서 데이터 파일 가져오기**

In [ ]:
#cell 03
!aws s3 cp s3://$bucket/$prefix/ . --recursive

Amazon S3에서 데이터를 다운로드한 후, Pandas DataFrame으로 로드할 수 있습니다

In [ ]:
# cell 04
from glob import glob
model_data = pd.concat([pd.read_csv(f) for f in glob("output*/*.csv")])
model_data = pd.get_dummies(model_data, dtype=int)

In [ ]:
model_data

In [ ]:
# cell 05
# 데이터를 무작위로 정렬한 후 처음 70%, 다음 20%, 마지막 10%로 분할
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])

Amazon SageMaker의 XGBoost 컨테이너는 libSVM 또는 CSV 데이터 형식을 기대합니다. 이 예제에서는 CSV를 사용하겠습니다. 첫 번째 열은 타겟 변수여야 하며 CSV에는 헤더가 포함되지 않아야 합니다. 또한 반복적이지만 train|validation|test 분할 후에 이 작업을 수행하는 것이 가장 쉽습니다. 이렇게 하면 무작위 재정렬로 인한 정렬 불일치 문제를 방지할 수 있습니다.

In [ ]:
# cell 06
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

이제 Amazon SageMaker의 관리형 훈련이 사용할 수 있도록 파일을 S3에 복사하겠습니다.

In [ ]:
# cell 07
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## End of Lab 1


---

## 훈련
이제 대부분의 특성이 왜곡된 분포를 가지고 있고, 일부는 서로 높은 상관관계를 가지며, 일부는 타겟 변수와 비선형 관계를 가지는 것을 알 수 있습니다. 또한 미래 잠재 고객을 타겟팅하기 위해서는 왜 그 잠재 고객이 타겟팅되었는지 설명하는 것보다 좋은 예측 정확도가 선호됩니다. 이러한 측면들을 종합하면 그래디언트 부스팅 트리가 좋은 후보 알고리즘이 됩니다.

알고리즘을 이해하는 데는 여러 복잡한 부분이 있지만, 높은 수준에서 보면 그래디언트 부스팅 트리는 많은 단순 모델의 예측을 결합하여 작동하며, 각 모델은 이전 모델의 약점을 해결하려고 시도합니다. 이렇게 함으로써 단순 모델의 집합이 실제로 크고 복잡한 모델을 능가할 수 있습니다. 다른 Amazon SageMaker 노트북에서는 그래디언트 부스팅 트리와 유사한 알고리즘과의 차이점에 대해 자세히 설명합니다.

`xgboost`는 그래디언트 부스팅 트리를 위한 매우 인기 있는 오픈소스 패키지입니다. 계산적으로 강력하고, 완전한 기능을 갖추고 있으며, 많은 머신러닝 경진대회에서 성공적으로 사용되었습니다. Amazon SageMaker의 관리형 분산 훈련 프레임워크를 사용하여 훈련된 간단한 `xgboost` 모델부터 시작하겠습니다.

먼저 Amazon SageMaker의 XGBoost 구현을 위한 ECR 컨테이너 위치를 지정해야 합니다.

In [ ]:
# cell 11
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

그런 다음 CSV 파일 형식으로 훈련하기 때문에, 훈련 함수가 S3의 파일에 대한 포인터로 사용할 수 있는 `s3_input`을 생성하고 콘텐츠 타입이 CSV임을 지정합니다.

In [ ]:
# cell 12
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

먼저 estimator에 훈련 파라미터를 지정해야 합니다. 여기에는 다음이 포함됩니다:
1. `xgboost` 알고리즘 컨테이너
1. 사용할 IAM 역할
1. 훈련 인스턴스 타입 및 개수
1. 출력 데이터를 위한 S3 위치
1. 알고리즘 하이퍼파라미터

그런 다음 `.fit()` 함수를 사용하여 다음을 지정합니다:
1. 출력 데이터를 위한 S3 위치. 이 경우 훈련 세트와 검증 세트가 모두 전달됩니다.

In [ ]:
# cell 13
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

---

## 호스팅
이제 데이터에 대해 `xgboost` 알고리즘을 훈련했으므로, 실시간 엔드포인트 뒤에서 호스팅되는 모델을 배포해보겠습니다.

In [ ]:
# cell 14
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m5.xlarge')

---

## 평가
머신러닝 모델의 성능을 비교하는 방법은 여러 가지가 있지만, 실제 값과 예측 값을 간단히 비교하는 것부터 시작하겠습니다. 이 경우, 고객이 정기예금에 가입했는지(`1`) 아닌지(`0`)를 단순히 예측하므로, 간단한 혼동 행렬을 생성합니다.

먼저 엔드포인트로 데이터를 전송하고 데이터를 받는 방법을 결정해야 합니다. 현재 데이터는 노트북 인스턴스의 메모리에 NumPy 배열로 저장되어 있습니다. HTTP POST 요청으로 전송하기 위해 CSV 문자열로 직렬화한 다음 결과 CSV를 디코딩하겠습니다.

*참고: CSV 형식으로 추론할 때, SageMaker XGBoost는 데이터에 타겟 변수가 포함되지 않아야 합니다.*

In [ ]:
# cell 15
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

이제 다음을 수행하는 간단한 함수를 사용하겠습니다:
1. 테스트 데이터셋을 반복
1. 행의 미니 배치로 분할
1. 미니 배치를 CSV 문자열 페이로드로 변환 (먼저 데이터셋에서 타겟 변수를 제거합니다)
1. XGBoost 엔드포인트를 호출하여 미니 배치 예측 검색
1. 예측을 수집하고 모델이 제공하는 CSV 출력을 NumPy 배열로 변환

In [ ]:
# cell 16
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(), xgb_predictor)

이제 혼동 행렬을 확인하여 실제 값 대비 예측이 얼마나 잘 되었는지 확인하겠습니다.

In [ ]:
# cell 17
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

약 6938명의 잠재 고객 중 3095명이 가입할 것으로 예측했고 그 중 2986명이 실제로 가입했습니다. 또한 예측하지 못했지만 가입한 414명의 가입자가 있었습니다. 이 결과를 개선하기 위해 모델을 조정할 수 있습니다(그리고 조정해야 합니다). 가장 중요한 것은 최소한의 노력으로 [여기](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf)에 게시된 것과 유사한 정확도를 달성했다는 점입니다.

_참고: 알고리즘의 서브샘플에 무작위성 요소가 있기 때문에 결과가 위의 텍스트와 약간 다를 수 있습니다._

### 정리
더 이상 사용하지 않을 이 노트북에서 생성한 리소스를 삭제합니다.

In [ ]:
# cell 28
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

--
## 서버리스 배포 (선택사항)

SageMaker는 요청이 없을 때 0으로 축소되는 엔드포인트를 사용하는 서버리스 구성도 지원합니다. 이 기능을 활성화하려면 엔드포인트에 대한 서버리스 구성을 설정해야 합니다. MaxConcurrency로 알려진 단일 엔드포인트의 현재 최대 동시 호출 수는 1에서 200 사이의 값이 될 수 있으며, MemorySize는 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB 또는 6144 MB 중 하나가 될 수 있습니다.

In [ ]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

xgb_serverless_predictor = xgb.deploy(
    serverless_inference_config=ServerlessInferenceConfig(
        memory_size_in_mb=2048,
        max_concurrency=1
    )
)

### 엔드포인트 호출
요청을 전송하여 엔드포인트를 호출합니다. 다음은 다이렉트 마케팅 데이터셋에서 가져온 샘플 데이터 포인트입니다.

In [ ]:
sample = validation_data.sample(n=5).drop(["y_yes", "y_no"], axis=1).values
len(sample[0])

In [ ]:
xgb_serverless_predictor.serializer = sagemaker.serializers.CSVSerializer()

xgb_serverless_predictor.predict(sample)

### 정리
더 이상 사용하지 않을 이 노트북에서 생성한 리소스를 삭제합니다.

In [ ]:
xgb_serverless_predictor.delete_endpoint()

## 자동 모델 튜닝 (선택사항)
하이퍼파라미터 튜닝이라고도 하는 Amazon SageMaker 자동 모델 튜닝은 지정한 알고리즘과 하이퍼파라미터 범위를 사용하여 데이터셋에서 많은 훈련 작업을 실행하여 모델의 최상의 버전을 찾습니다. 그런 다음 선택한 메트릭으로 측정했을 때 가장 우수한 성능을 발휘하는 모델을 생성하는 하이퍼파라미터 값을 선택합니다.
예를 들어, 이 마케팅 데이터셋에서 이진 분류 문제를 해결하려고 한다고 가정해 보겠습니다. 목표는 XGBoost 알고리즘 모델을 훈련하여 알고리즘의 곡선 아래 면적(auc) 메트릭을 최대화하는 것입니다. 최상의 모델을 훈련하는 데 사용할 eta, alpha, min_child_weight 및 max_depth 하이퍼파라미터의 값을 모릅니다. 이러한 하이퍼파라미터의 최상의 값을 찾기 위해 Amazon SageMaker 하이퍼파라미터 튜닝이 검색할 값의 범위를 지정하여 선택한 목표 메트릭으로 측정했을 때 가장 우수한 성능을 발휘하는 훈련 작업을 생성하는 값의 조합을 찾을 수 있습니다. 하이퍼파라미터 튜닝은 지정한 범위의 하이퍼파라미터 값을 사용하는 훈련 작업을 시작하고 가장 높은 auc를 가진 훈련 작업을 반환합니다.


In [ ]:
# cell 18
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}


In [ ]:
# cell 19
objective_metric_name = 'validation:auc'

In [ ]:
# cell 20
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)


In [ ]:
# cell 21
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
# cell 22
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# cell 23
# 최상의 훈련 작업 이름 반환
tuner.best_training_job()

In [ ]:
# cell 24
# 최상의 훈련된 모델 또는 사용자 지정 모델을 Amazon SageMaker 엔드포인트에 배포
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# cell 25
# 직렬화기 생성
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# cell 26
# 예측
predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(),tuner_predictor)

In [ ]:
# cell 27
# 예측을 수집하고 모델이 제공하는 CSV 출력을 NumPy 배열로 변환
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

---

## 확장

이 예제는 비교적 작은 데이터셋을 분석했지만, 분산 관리형 훈련 및 실시간 모델 호스팅과 같은 Amazon SageMaker 기능을 활용했으며, 이는 훨씬 더 큰 문제에도 쉽게 적용할 수 있습니다. 예측 정확도를 더욱 향상시키기 위해 거짓 양성과 거짓 음성의 혼합을 변경하기 위해 예측 임계값을 조정하거나 하이퍼파라미터 튜닝과 같은 기술을 탐색할 수 있습니다. 실제 시나리오에서는 수동으로 특성을 엔지니어링하는 데 더 많은 시간을 할애하고 초기 데이터셋에서 사용할 수 없는 고객 정보가 포함된 추가 데이터셋을 찾을 것입니다.

### 정리

이 노트북 작업을 완료했다면 아래 셀을 실행하세요. 생성한 호스팅 엔드포인트가 제거되고 남아있는 인스턴스로 인한 요금이 발생하지 않습니다.

In [ ]:
# cell 29
tuner_predictor.delete_endpoint(delete_endpoint_config=True)